# Wolfram Eropean Call Option example

This example demonstrates a contract utilizing Wolfram's computational intelligence as an Oracle for a European Call derivative.

**1:3 linear bet on BTCADA ticker**

**INTRODUCTION**

Alice is betting `premium` amount on: price of Bitcoin to be MORE than `strike + premium`. 

*How: Alice buys a call option, Alice offers a put to Bob*

Bob is betting `margin` amount on: price to be LESS than `strike + premium`.

*How: Bob offers a call to Alice, Bob buys a put option (with premium = alice_margin, margin = alice_premium, strike at net profit=0)*

**MECHANICS**

The option’s bet is ternary (risk is “linearly amortized” between total win and total loss, See Picture 1).

Parties are “splitting the pie”, in contrast to “winner takes all” agreement, as was the case in lesson 1.

*Note “simple bet” from lesson 1 could be seen as a binary option (an option with infinite leverage).*

Since P2P option has collar-like collateralization:
- If Alice wins, she’s gonna net profit `min(value-premium, margin)` (capped at margin for Alice)
- If Bob wins, he’ll net profit `min(premium-value, premium)` (capped at premium for bob)

Since the option is a right to buy an asset at strike price, the gross value (not accounting for premium’s bias) of an option is:

`value = notional * (market - strike).` 

**EXTRA TERMS:**

Tender settlement.

At the money (net profit at t0 is 0)

*(strike is price minus “premium divided by notional” at the purchase date, so the contract starts at the threshold between loss and profit)*

European call: can only be exercised at certain date.

[MARLOWE CONTRACT HERE](wolfram-eurocall.marlowe)

[Marlowe Playground](https://play.marlowe.iohk.io//)

![payoff curve](Collar-300x178.gif)

Picture 1. Option's payoff curve (premium = margin). For Alice, loss potential is limited by premium, and profit by margin. Vice versa for Bob. [Source](https://celtinvest.com/collar/)

*Note: The slope of the “put strike -> call strike” line corresponds to leverage*

————————————————————————————————————————————————————————————

Quantitative notes:
- From “value at risk” perspective, `premium` could be seen as Alice’s collateral while `margin` is Bob’s collateral
- Swapping `market` and `strike` in the `value` formula would convert "Call" into a “Put” option
- Additionally swapping `margin`s and `premium`s would allow create a “perfect hedge“ pair of call and put option
- Due to tender settlement, `notional` only represents leverage, not the actual asset
- If `notional` = 1 BTC, then deal is non-leveraged (1X). This would be equivalent to spot trading, but with protected portfolio and lower collateral (the whole notional would become collateral then).
- Tender settlement allows speculating parties to avoid collateralization of `notional` amount of BTC and `notional * strike` amount of ADA. Physical settlement would not require Oracle, since parties would just swap (optionally) their collaterals (ADA for BTC).

Technical notes
- `Choice` "dummy" enforces minimum exercise time (a hack for Marlowe), in reality its address should be derived from randomly generated public (not private!) key
- Oracle is responsible for reporting no later than maximum exercise time

Test cases (for Marlowe Playground):

```
Terms: Notional = 1, strike = 1000, premium = 100, margin = 300

observed price        -> Alice payout       , Bob payout 
————————————————————————————————————————————————————————————
1100                  ->  100               , 300
100000                ->  400               , 0
0                     ->  0                 , 400
1110                  ->  110               , 290
1090                  ->  90                , 310



Terms: Notional = 2 (2X leverage), strike = 1000, premium = 100, margin = 300

observed price        -> Alice payout, Bob payout 
————————————————————————————————————————————————————————————
1100                  ->  100                , 300
100000                ->  400                , 0
0                     ->  0                  , 400
1110                  ->  120                , 280
1090                  ->  80                 , 320
```

## Preliminaries

See [Lesson 0. Preliminaries](00-preliminaries.md) for information on setting up one's environment for using this tutorial.

The lesson assumes that the following environment variables have been set.
- `CARDANO_NODE_SOCKET_PATH`: location of Cardano node's socket.
- `CARDANO_TESTNET_MAGIC`: testnet magic number.

It also assumes that the Lender and Borrower parties have addresses, signing keys, and funds.
- Lender
    - [keys/alice.address](keys/alice.address): Cardano address for Alice
    - [keys/alice.skey](keys/alice.skey): location of signing key file for Alice
- Borrower
    - [keys/bob.address](keys/borrower.address): Cardano address for Bob
    - [keys/bob.skey](keys/borrower.skey): location of signing key file for Bob

### Access to Cardano node and Marlowe Runtime

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. The followind commands will set the required environment variables to use a local docker deployment on the default ports. It will also set some supplementary environment variables.

In [1]:
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=2 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"

SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes
ADA=1000000

CARDANO_NODE_SOCKET_PATH = /ipc/node.socket
CARDANO_TESTNET_MAGIC = 2


# Update contract deposit deadline, exercise and expiration dates


In [5]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes
NOW="$((`date -u +%s` * SECOND))"
echo NOW = "$NOW" POSIX milliseconds = "`date -d @$((NOW / SECOND))`"
ADA=1000000

DEPOSIT_PERIOD=$((10 * MINUTE)) # Deposit within 10 minutes
PERIOD_TO_EXERCISE=$((10 * MINUTE)) # Oracle can only report price 15 minutes after now
PERIOD_TO_REPORT_OBSERVATION=$((1000 * MINUTE)) # Contract will expire in 30 minutes

TIME1=$((NOW + DEPOSIT_PERIOD))
TIME2=$((NOW + DEPOSIT_PERIOD + 1))
TIME11=$((NOW + DEPOSIT_PERIOD + PERIOD_TO_EXERCISE))
TIME12=$((NOW + DEPOSIT_PERIOD + PERIOD_TO_EXERCISE + PERIOD_TO_REPORT_OBSERVATION))

REPLACE1='s/1000000000000001/'$TIME1'/g'
REPLACE2='s/1000000000000002/'$TIME2'/g'
REPLACE11='s/1000000100000001/'$TIME11'/g'
REPLACE12='s/1000000100000002/'$TIME12'/g'

echo $REPLACE1 "|||" $REPLACE2 "|||" $REPLACE11 "|||" $REPLACE12

cat 'wolfram-eurocall-template.json' | sed $REPLACE1 | sed $REPLACE2 | sed $REPLACE11 | sed $REPLACE12 > wolfram-eurocall.json 

echo DEPOSIT DEADLINE = "$NOW" POSIX milliseconds = "`date -d @$((TIME1 / SECOND))`"
echo EXERCISE DATE = "$TIME11" POSIX milliseconds = "`date -d @$((TIME11 / SECOND))`"
echo EXPIRATION DATE = "$TIME12" POSIX milliseconds = "`date -d @$((TIME12 / SECOND))`"


NOW = 1687380031000 POSIX milliseconds = Wed Jun 21 08:40:31 PM UTC 2023
s/1000000000000001/1687380631000/g ||| s/1000000000000002/1687380631001/g ||| s/1000000100000001/1687381231000/g ||| s/1000000100000002/1687441231000/g
DEPOSIT DEADLINE = 1687380031000 POSIX milliseconds = Wed Jun 21 08:50:31 PM UTC 2023
EXERCISE DATE = 1687381231000 POSIX milliseconds = Wed Jun 21 09:00:31 PM UTC 2023
EXPIRATION DATE = 1687441231000 POSIX milliseconds = Thu Jun 22 01:40:31 PM UTC 2023


Note the test network magic number:
- `preprod` = 1
- `preview` = 2

# Examine accounts
See [Lesson 0. Preliminaries](00-preliminaries.md) to request test ADA

# Check funds

Check that an address and key has been created. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

### Alice's address and funds

Check that an address and key has been created for Alice. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [48]:
ALICE_SKEY=keys/alice.skey
ALICE_ADDR=$(cat keys/alice.address)
echo "ALICE_ADDR = $ALICE_ADDR"

ALICE_ADDR = addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq


Check that Alicer has at least one hundred ada.

In [16]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ALICE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5c5328866aa0cf7af63ffbb68b5c3a99a4cf993c56d682be8b62fcf97fcc07f9     0        75683649 lovelace + TxOutDatumNone
bd58fa8f26eeb65417080126b87a3be56dae66639dde5494bc00bbd7c49278f1     1        1000000000 lovelace + TxOutDatumNone
d6ffcd7d0dceba4236391544cd4ada8842f32878395cd088991ee2e1fe73b01c     1        1000000000 lovelace + TxOutDatumNone
d9096dd67b85d73576a5a8a3135a08dd1be1f6e26b0cbee547327b53b530b82b     0        796848386 lovelace + TxOutDatumNone
e941faa76c97cb1956d2e50ac4dc5408e0f92e355ba8e80d480ffa3a2859eb5e     0        816333124 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [17]:
echo "$EXPLORER_URL"/address/"$ALICE_ADDR"

https://preview.cardanoscan.io/address/addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq


### Bob's address and funds

Check that an address and key has been created for Bob. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [49]:
BOB_SKEY=keys/bob.skey
BOB_ADDR=$(cat keys/bob.address)
echo "BOB_ADDR = $BOB_ADDR"

BOB_ADDR = addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf


Check that the borrower has at least one hundred ada.

In [19]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BOB_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
37cc9570bd16281f230bd3dd1d54b43c9b6a0861692d7cc1984fe5aa9a0e642f     0        286424194 lovelace + TxOutDatumNone
77cbdf3d315ca6f6ab33adf708a511b88e269e53227f45d754d9f284833e4d84     0        78306842 lovelace + TxOutDatumNone
84b42a912f1410b7d9c5de30e145960daba47de1476760a06b27e2b4fc49e3fd     0        399777783 lovelace + TxOutDatumNone
d6ffcd7d0dceba4236391544cd4ada8842f32878395cd088991ee2e1fe73b01c     2        1000000000 lovelace + TxOutDatumNone
dcfbdc54d2f294cefddf5ddb8c81acf7c314cfba7e18759a9b49813db77b284d     0        907632079 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [20]:
echo "$EXPLORER_URL"/address/"$BOB_ADDR"

https://preview.cardanoscan.io/address/addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf


## Examine the contract

View the contract file as YAML.

In [10]:
json2yaml wolfram-eurocall.json

then:
  then:
    then:
      then:
        then:
          when:
          - then:
              when:
              - then:
                  when:
                  - then: close
                    case:
                      for_choice:
                        choice_owner:
                          address: invalid1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j
                        choice_name: DO NOT ENTER, SKIP TO NEXT TIMEOUT INSTEAD
                      choose_between:
                      - to: 0
                        from: 0
                  timeout_continuation:
                    when:
                    - then:
                        then:
                          then:
                            then:
                              then: close
                              if:
                                value:
                                  use_value: value_ada
                                le_than: 0
                              else:
    

### \[Optional, but recommended\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe.iohk.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://https://play.marlowe.iohk.io//) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

See [Lesson 1](01-runtime-cli.ipynb) for an example of performing step 6.

# Execute transactions

Check that an address and key has been created. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

## Transaction 1. Create the Contract

Marlowe CLI\'s command `marlowe-cli run initialize` will build the creation information for a Marlowe contract. We provide it the JSON files containing the contract and initial state. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

In [6]:
marlowe-cli run initialize \
  --permanently-without-staking \
  --contract-file wolfram-eurocall.json \
  --state-file wolfram-state.json \
  --out-file eurocall-1.json

We now use Marlowe CLI\'s `marlowe-cli run auto-execute` command to construct and submit the creation transaction.

In [7]:
TX_1=$(
marlowe-cli run auto-execute \
  --marlowe-out-file eurocall-1.json \
  --change-address "$ALICE_ADDR" \
  --required-signer "$ALICE_SKEY" \
  --out-file eurocall-tx-1.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"


Fee: Lovelace 242633
Size: 1587 / 16384 = 9%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX_1 = 0a66b44816edbf0d1248c7f09e528ed7e94c91e138b1c6d2ea2011cd0f0cd4a9


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [105]:
echo "$EXPLORER_URL"/transaction/"$TX_1?tab=utxo"

https://preview.cardanoscan.io/transaction/e8efde347dfb09e159b37d37586e35ab3276f735adb03118b6631f98b8a12c22?tab=utxo


One can also examine the contract's UTxO using `cardano-cli`.

In [106]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_1#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
e8efde347dfb09e159b37d37586e35ab3276f735adb03118b6631f98b8a12c22     1        600000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "0bdc17570003b2ab0d48761a8b1bb46e0a4612e7e321087e65752005055a4a83"


## Skip LET definitions

In [8]:
marlowe-cli run prepare \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 1 * MINUTE))" \
  --marlowe-file eurocall-1.json \
  --out-file eurocall-2.json

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1687379553000},POSIXTime {getPOSIXTime = 1687379673999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1687379553000},POSIXTime {getPOSIXTime = 1687379673999}), txInputs = []}


In [9]:
TX_2=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_1#1" \
  --marlowe-in-file eurocall-1.json \
  --marlowe-out-file eurocall-2.json \
  --change-address "$ALICE_ADDR" \
  --required-signer "$ALICE_SKEY" \
  --out-file eurocall-tx-2.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"


Fee: Lovelace 868456
Size: 2951 / 16384 = 18%
Execution units:
  Memory: 7297355 / 14000000 = 52%
  Steps: 2034463110 / 10000000000 = 20%
TX_2 = ef1c107f17ee3a545aa1660a50ccf44aa257774eda4347761bffff7222c3f496


## Transaction 2. Alice deposit

Alice is going to bet some valuable cash she was saving for a rainy day

In [10]:
marlowe-cli run prepare \
  --deposit-account "$BOB_ADDR" \
  --deposit-party "$ALICE_ADDR" \
  --deposit-amount "100000000" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 1 * MINUTE))" \
  --marlowe-file eurocall-2.json \
  --out-file eurocall-3.json

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1687379575000},POSIXTime {getPOSIXTime = 1687379695999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1687379575000},POSIXTime {getPOSIXTime = 1687379695999}), txInputs = [NormalInput (IDeposit "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\"" "\"addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq\"" (Token "" "") 100000000)]}


In [17]:
TX_3=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_2#1" \
  --marlowe-in-file eurocall-2.json \
  --marlowe-out-file eurocall-3.json \
  --change-address "$ALICE_ADDR" \
  --required-signer "$ALICE_SKEY" \
  --out-file eurocall-tx-3.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"


Fee: Lovelace 837366
Size: 2809 / 16384 = 17%
Execution units:
  Memory: 6988142 / 14000000 = 49%
  Steps: 1937379854 / 10000000000 = 19%
TX_3 = 9aae31fa7c1e5d2366976aa570973f3678fdf1e444d5684313ff208e102408fc


In [27]:
echo "$EXPLORER_URL"/transaction/"$TX_2?tab=utxo"

https://preview.cardanoscan.io/transaction/731d8ced76f3d1e97381d449b54f54418b945a3909a49b3f4fa394a0710c7755?tab=utxo


In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_3?tab=utxo"

https://preview.cardanoscan.io/transaction/731d8ced76f3d1e97381d449b54f54418b945a3909a49b3f4fa394a0710c7755?tab=utxo


In [39]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ALICE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5c5328866aa0cf7af63ffbb68b5c3a99a4cf993c56d682be8b62fcf97fcc07f9     0        75683649 lovelace + TxOutDatumNone
63b44779d26e430be7654d3cb96007e640340c1f8d8add5400486eedc82028c9     0        798544566 lovelace + TxOutDatumNone
d6ffcd7d0dceba4236391544cd4ada8842f32878395cd088991ee2e1fe73b01c     1        1000000000 lovelace + TxOutDatumNone
d9096dd67b85d73576a5a8a3135a08dd1be1f6e26b0cbee547327b53b530b82b     0        796848386 lovelace + TxOutDatumNone
e941faa76c97cb1956d2e50ac4dc5408e0f92e355ba8e80d480ffa3a2859eb5e     0        816333124 lovelace + TxOutDatumNone


In [40]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
63b44779d26e430be7654d3cb96007e640340c1f8d8add5400486eedc82028c9     1        700000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "072cbca1b638831001626ab64306bc09df49d7b6366614dde6a5cc6002c3346f"


## Transaction 3. Bob deposit

Bob is unemployed because of his gampling problems. This bet is his only chance to find a good therapist.

In [11]:
marlowe-cli run prepare \
  --deposit-account "$BOB_ADDR" \
  --deposit-party "$BOB_ADDR" \
  --deposit-amount "300000000" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file eurocall-3.json \
  --out-file eurocall-4.json

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1687379582000},POSIXTime {getPOSIXTime = 1687379942999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1687379582000},POSIXTime {getPOSIXTime = 1687379942999}), txInputs = [NormalInput (IDeposit "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\"" "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\"" (Token "" "") 300000000)]}


In [19]:
TX_4=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_3#1" \
  --marlowe-in-file eurocall-3.json \
  --marlowe-out-file eurocall-4.json \
  --change-address "$BOB_ADDR" \
  --required-signer "$BOB_SKEY" \
  --out-file eurocall-tx-4.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"


Fee: Lovelace 851670
Size: 2558 / 16384 = 15%
Execution units:
  Memory: 7220466 / 14000000 = 51%
  Steps: 1979742567 / 10000000000 = 19%
TX_4 = 8946ed2ac3f1d8a0815fb0faff49468a5dfe5eaaf06c8362f3edbe34bc209976


In [47]:
echo "$EXPLORER_URL"/transaction/"$TX_4?tab=utxo"

/transaction/5954d646c4065186fa4a726c20b217bf7f7c4fdc5d83da96bc238e962a9ee97d?tab=utxo


# Progress prematurely AND FAIL

In [3]:
marlowe-cli run prepare \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file eurocall-4.json \
  --out-file FAIL-eurocall-5.json


Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1687379897000},POSIXTime {getPOSIXTime = 1687380257999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1687379897000},POSIXTime {getPOSIXTime = 1687380257999}), txInputs = []}
TEUselessTransaction


: 1

# Run Mini-Oracle on time

In [10]:
# reminder
NEWNOW="$((`date -u +%s` * SECOND))"
echo NOW = "$NEWNOW" POSIX milliseconds = "`date -d @$((NEWNOW / SECOND))`"
echo EXERCISE DATE = "$TIME11" POSIX milliseconds = "`date -d @$((TIME11 / SECOND))`"

NOW = 1687381261000 POSIX milliseconds = Wed Jun 21 09:01:01 PM UTC 2023
EXERCISE DATE = 1687381231000 POSIX milliseconds = Wed Jun 21 09:00:31 PM UTC 2023


In [33]:
# now this will work
marlowe-cli run prepare \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 1 * MINUTE))" \
  --marlowe-file eurocall-4.json \
  --out-file eurocall-5.json


Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1687384675000},POSIXTime {getPOSIXTime = 1687384795999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1687384675000},POSIXTime {getPOSIXTime = 1687384795999}), txInputs = []}


In [35]:
TICKER=$(curl "https://api.wolframalpha.com/v1/result?appid=6WU6JX-46EP5U9AGX&i=1%20btc%20to%20usd%20number" | sed 's/[^0-9]//g')
echo "TICKER = $TICKER"

marlowe-cli run prepare \
  --choice-name "WOLF_BTCADA" \
  --choice-party "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j" \
  --choice-number "$TICKER" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 1 * MINUTE))" \
  --marlowe-file eurocall-5.json \
  --out-file eurocall-6.json


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22    0    22    0     0     29      0 --:--:-- --:--:-- --:--:--    29
TICKER = 30200
Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1687384693000},POSIXTime {getPOSIXTime = 1687384813999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1687384693000},POSIXTime {getPOSIXTime = 1687384813999}), txInputs = [NormalInput (IChoice (ChoiceId "WOLF_BTCADA" "\"addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j\"") 30200)]}
Payment 1
  Acccount: "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\""
  Payee: Account "\"addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq\""
  Ada: Lovelace {getLovelace = 700000000}
Payment 2
  Acccount: "\"addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq\""
  Payee: Party "\"addr_test1vrc79wasc0na5zglc

### **Unfortunatelly, Marlowe breaks its own semantics on-chain, when it comes to timeouts**

What message means is that: The contract moved in one state in marlowe-prepare, but ended in another on-chain

In [36]:
TX_5=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_4#1" \
  --marlowe-in-file eurocall-4.json \
  --marlowe-out-file eurocall-5.json \
  --change-address "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j" \
  --required-signer "keys/bob.skey" \
  --out-file tx-5.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_5 = $TX_5"

TxBodyScriptExecutionError [(ScriptWitnessIndexTxIn 0,ScriptErrorWrongDatum "b696981d1615ce7278b5203ec99125fd464d36525303a44f4488aed0f324dd6e")]
TX_5 = 


---------------------------------------------
Just to get some satisfaction. Here's simplified version of Eurocall, that works. 
It doesn't have native Exercise Date limitations, so responsibility goes to the Oracle.


```
When
    [Case
        (Deposit
            (Address "addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq")
            (Address "addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq")
            (Token "" "")
            (Constant 100000000)
        )
        (When
            [Case
                (Deposit
                    (Address "addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf")
                    (Address "addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf")
                    (Token "" "")
                    (Constant 300000000)
                )
                (When
                    [Case
                        (Choice
                            (ChoiceId
                                "WOLF_BTCADA"
                                (Address "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j")
                            )
                            [Bound 0 1000000]
                        )
                        (Pay
                            (Address "addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq")
                            (Account (Address "addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf"))
                            (Token "" "")
                            (Constant 100000000)
                            (If
                                (ValueLE
                                    (SubValue
                                        (ChoiceValue
                                            (ChoiceId
                                                "WOLF_BTCADA"
                                                (Address "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j")
                                            ))
                                        (Constant 1000)
                                    )
                                    (Constant 0)
                                )
                                Close 
                                (Pay
                                    (Address "addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf")
                                    (Account (Address "addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq"))
                                    (Token "" "")
                                    (MulValue
                                        (Constant 1000000)
                                        (SubValue
                                            (ChoiceValue
                                                (ChoiceId
                                                    "WOLF_BTCADA"
                                                    (Address "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j")
                                                ))
                                            (Constant 1000)
                                        )
                                    )
                                    Close 
                                )
                            )
                        )]
                    1000000100000002 Close 
                )]
            1000000000000002 Close 
        )]
    1000000000000001 Close 
```

In [59]:
mkdir chain
mkdir chain/simple

marlowe-cli run initialize \
  --permanently-without-staking \
  --contract-file wolfram-eurocall-simple.json \
  --state-file wolfram-state.json \
  --out-file ./chain/simple/eurocall-1.json

marlowe-cli run prepare \
  --deposit-account "$BOB_ADDR" \
  --deposit-party "$ALICE_ADDR" \
  --deposit-amount "100000000" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file ./chain/simple/eurocall-1.json \
  --out-file ./chain/simple/eurocall-2.json

marlowe-cli run prepare \
  --deposit-account "$BOB_ADDR" \
  --deposit-party "$BOB_ADDR" \
  --deposit-amount "300000000" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file ./chain/simple/eurocall-2.json \
  --out-file ./chain/simple/eurocall-3.json

TICKER=$(curl "https://api.wolframalpha.com/v1/result?appid=6WU6JX-46EP5U9AGX&i=1%20btc%20to%20usd%20number" | sed 's/[^0-9]//g')
echo "TICKER = $TICKER"

marlowe-cli run prepare \
  --choice-name "WOLF_BTCADA" \
  --choice-party "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j" \
  --choice-number "$TICKER" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file ./chain/simple/eurocall-3.json \
  --out-file ./chain/simple/eurocall-4.json


TXX_1=$(
marlowe-cli run auto-execute \
  --marlowe-out-file ./chain/simple/eurocall-1.json \
  --change-address "$ALICE_ADDR" \
  --required-signer "$ALICE_SKEY" \
  --out-file ./chain/simple/eurocall-tx-1.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TXX_1"

TXX_2=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TXX_1#1" \
  --marlowe-in-file ./chain/simple/eurocall-1.json \
  --marlowe-out-file ./chain/simple/eurocall-2.json \
  --change-address "$ALICE_ADDR" \
  --required-signer "$BOB_SKEY" \
  --out-file ./chain/simple/eurocall-tx-2.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TXX_2"

TXX_3=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TXX_2#1" \
  --marlowe-in-file ./chain/simple/eurocall-2.json \
  --marlowe-out-file ./chain/simple/eurocall-3.json \
  --change-address "$ALICE_ADDR" \
  --required-signer "$BOB_SKEY" \
  --out-file ./chain/simple/eurocall-tx-3.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TXX_3"

TXX_4=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TXX_3#1" \
  --marlowe-in-file ./chain/simple/eurocall-3.json \
  --marlowe-out-file ./chain/simple/eurocall-4.json \
  --change-address "$ALICE_ADDR" \
  --required-signer "keys/oracle.skey" \
  --out-file ./chain/simple/eurocall-tx-4.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TXX_4"


mkdir: cannot create directory ‘chain’: File exists
mkdir: cannot create directory ‘chain/simple’: File exists
Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1687388772000},POSIXTime {getPOSIXTime = 1687389132999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1687388772000},POSIXTime {getPOSIXTime = 1687389132999}), txInputs = [NormalInput (IDeposit "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\"" "\"addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq\"" (Token "" "") 100000000)]}
TEApplyNoMatchError
Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1687388773000},POSIXTime {getPOSIXTime = 1687389133999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1687388773000},POSIXTime {getPOSIXTime = 1687389133999}), txInputs = [NormalInput (IDeposit "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\"" "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89n

In [57]:
echo "$EXPLORER_URL"/transaction/"$TXX_1?tab=utxo"
echo "$EXPLORER_URL"/transaction/"$TXX_2?tab=utxo"
echo "$EXPLORER_URL"/transaction/"$TXX_3?tab=utxo"
echo "$EXPLORER_URL"/transaction/"$TXX_4?tab=utxo"

https://preview.cardanoscan.io/transaction/b3253a7b9af92ec94d70896b2f65cf1e993de6404b9f0ba086bf5c403517058a?tab=utxo
https://preview.cardanoscan.io/transaction/?tab=utxo
https://preview.cardanoscan.io/transaction/?tab=utxo
https://preview.cardanoscan.io/transaction/?tab=utxo
